In [1]:
import os
import torch
from PIL import Image
from transformers import AutoTokenizer
from groundingdino.util.utils import clean_state_dict
from groundingdino.models import build_model
from groundingdino.util.slconfig import SLConfig
from torchvision import transforms
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches

/home/km/pytorch_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/km/pytorch_env/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [3]:
# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Paths to your model and configuration
model_path = "/home/km/content/output_tiny/checkpoint0014.pth"
config_path = "/home/km/content/Open-GroundingDino/config/cfg_odvg.py"

In [4]:
# Function to load the model
def load_model(config_path, checkpoint_path):
    args = SLConfig.fromfile(config_path)
    model = build_model(args)
    checkpoint = torch.load(checkpoint_path, map_location="cpu")
    model.load_state_dict(clean_state_dict(checkpoint["model"]), strict=False)
    return model

# Load the model
model = load_model(config_path, model_path)
model = model.to(device)
model.eval()

/home/km/pytorch_env/lib/python3.10/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


/tmp/ipykernel_20244/3100025926.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location="cpu")


GroundingDINO(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-5): 6 x DeformableTransformerEncoderLayer(
          (self_attn): MultiScaleDeformableAttention(
            (sampling_offsets): Linear(in_features=256, out_features=256, bias=True)
            (attention_weights): Linear(in_features=256, out_features=128, bias=True)
            (value_proj): Linear(in_features=256, out_features=256, bias=True)
            (output_proj): Linear(in_features=256, out_features=256, bias=True)
          )
          (dropout1): Dropout(p=0.0, inplace=False)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (dropout2): Dropout(p=0.0, inplace=False)
          (linear2): Linear(in_features=2048, out_features=256, bias=True)
          (dropout3): Dropout(p=0.0, inplace=False)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_aff

In [5]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("/home/km/content/bert")

# Text prompt/prompts
text_prompts = ["nodule"]

# Preprocess text prompts
text_inputs = tokenizer(text_prompts, padding='longest', return_tensors="pt").to(device)

In [8]:
import torchvision.transforms as T
from groundingdino.datasets.transforms import Normalize

# Image Path
image_path = "/home/km/content/imgs/nod.jpeg"
image_pil = Image.open(image_path).convert("RGB")

transform = T.Compose([
    T.ToTensor(),
    Normalize(
        mean=model.pixel_mean,
        std=model.pixel_std
    )
])
image = transform(image_pil).unsqueeze(0).to(device)

AttributeError: 'GroundingDINO' object has no attribute 'pixel_mean'

In [4]:
!python "/home/km/content/Open-GroundingDino/tools/inference_on_a_image.py" \
  -c "/home/km/content/Open-GroundingDino/tools/GroundingDINO_SwinT_OGC.py" \
  -p "/home/km/content/output_tiny/checkpoint0014.pth" \
  -i "/home/km/content/imgs/node.jpg" \
  -t "nodule " \
  -o "/home/km/content/test/"

/home/km/pytorch_env/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/km/pytorch_env/lib/python3.10/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
final text_encoder_type: bert-base-uncased
/home/km/content/Open-GroundingDino/tools/inference_on_a_image.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/